In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from random import randint
import xlrd

Abadia B, Suñen I, Calvo P, Bartol F, Verdes G, Ferreras A (2018) Choroidal thickness measured using swept-source optical coherence tomography is reduced in patients with type 2 diabetes. PLoS ONE 13(2): e0191977. https://doi.org/10.1371/journal.pone.0191977

link to the dataset: https://figshare.com/articles/dataset/Choroidal_thickness_measured_using_swept-source_optical_coherence_tomography_is_reduced_in_patients_with_type_2_diabetes/5850960

In [28]:
data = pd.read_excel('Abadia2018.xlsx')

1) dataset name: Abadia2018
target: Years DM
URL: https://figshare.com/articles/dataset/Choroidal_thickness_measured_using_swept-source_optical_coherence_tomography_is_reduced_in_patients_with_type_2_diabetes/5850960


In [29]:
data.head()

,N,Age,Eye (right/left),Visual acuity,Baseline IOP,Group (1= control; 2=DM without DR; 3=mild NPDR; 4=moderate NPDR; 5=severe NPDR; 6= PDR),EMD (0=no; 1= sí),Accurate automatic segmentation (0=no; 1=yes),Quality,SF,...,N 1500µm,N 2000µm,N 2500µm,T 500µm,T 1000µm,T 1500µm,T 2000µm,T 2500µm,HbA1c,Years DM
0,1,70,left,0.6,20,4,1.0,1,89,110,...,92,89,100,147,168,181,183,134,7.6,32.00
1,2,70,right,0.8,22,4,0.0,1,95,186,...,121,89,73,163,165,186,183,157,7.6,32.00
2,3,75,right,0.4,14,4,1.0,1,89,79,...,157,155,160,76,100,110,131,107,7.5,33.00
3,4,75,left,0.6,14,4,0.0,1,95,228,...,249,254,239,183,163,155,163,160,7.5,33.00
4,5,66,left,0.5,16,4,1.0,1,94,81,...,63,87,73,81,84,102,100,87,8.0,13.85


In [30]:
data.columns

Index(['N', 'Age', 'Eye (right/left)', 'Visual acuity', 'Baseline IOP',
       'Group (1= control; 2=DM without DR; 3=mild NPDR; 4=moderate NPDR; 5=severe NPDR; 6= PDR)',
       'EMD (0=no; 1= sí)', 'Accurate automatic segmentation (0=no; 1=yes)',
       'Quality', 'SF', 'N 500µm', 'N 1000µm', 'N 1500µm', 'N 2000µm',
       'N 2500µm', 'T 500µm', 'T 1000µm', 'T 1500µm', 'T 2000µm', 'T 2500µm',
       'HbA1c', 'Years DM'],
      dtype='object')

In [31]:
data.drop(['N'], axis = 1, inplace = True)

In [32]:
data.isnull().values.any()

True

### I will have to map the strings on Eyes right/left becuase the iterative imputer only takes floats or integers

In [33]:
data.replace({'left': 0, 'right': 1}, inplace = True)

### Now we can use the iterative imputer to fill the nan values

In [34]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
iterative_imp = IterativeImputer()

In [35]:
iterative_imp.fit(data)

IterativeImputer(add_indicator=False, estimator=None,
                 imputation_order='ascending', initial_strategy='mean',
                 max_iter=10, max_value=None, min_value=None,
                 missing_values=nan, n_nearest_features=None, random_state=None,
                 sample_posterior=False, skip_complete=False, tol=0.001,
                 verbose=0)

In [36]:
data1 = iterative_imp.fit_transform(data)
data1 = pd.DataFrame(data=data1, columns = data.columns,)

The data is "duplicated" as each patient has 2 eyes, and therefore 2 entries in the dataframe. This can (and probably will) artificially inflate the r*2 predictor, and make the non-eye-related variables more important than they are when predicting for duration of diabetes.
 Since the entries are added consecutively, I will just take the average for each pair of 2 consecutive rows. This means, that I will only leave 1 row for each patient. 
 Also standart linear regression takes the assumption of independence between observations, we would be violating this assumption if we leave it as it is.

In [37]:
idx = len(data1) - 1 if len(data1) % 2 else len(data1)

In [38]:
datafinal = data1[:idx].groupby(data1.index[:idx] // 2).mean()

In [39]:
datafinal.head()

,Age,Eye (right/left),Visual acuity,Baseline IOP,Group (1= control; 2=DM without DR; 3=mild NPDR; 4=moderate NPDR; 5=severe NPDR; 6= PDR),EMD (0=no; 1= sí),Accurate automatic segmentation (0=no; 1=yes),Quality,SF,N 500µm,...,N 1500µm,N 2000µm,N 2500µm,T 500µm,T 1000µm,T 1500µm,T 2000µm,T 2500µm,HbA1c,Years DM
0,70.0,0.5,0.70,21.0,4.0,0.5,1.0,92.0,148.0,150.5,...,106.5,89.0,86.5,155.0,166.5,183.5,183.0,145.5,7.6,32.00
1,75.0,0.5,0.50,14.0,4.0,0.5,1.0,92.0,153.5,182.0,...,203.0,204.5,199.5,129.5,131.5,132.5,147.0,133.5,7.5,33.00
2,66.0,0.5,0.65,16.0,4.0,0.5,1.0,94.5,90.5,77.5,...,65.5,72.5,65.5,87.5,82.5,87.5,83.0,79.0,8.0,13.85
3,70.0,0.5,0.95,14.0,3.5,0.5,0.5,90.5,120.5,120.0,...,112.5,95.5,114.0,133.5,130.5,188.0,144.0,123.0,5.5,11.00
4,65.0,0.5,0.50,18.0,6.0,0.5,1.0,85.0,174.5,173.0,...,162.5,150.5,122.0,182.0,165.0,141.5,155.0,174.5,8.6,41.00


### Let's predict duration of diabetes

In [63]:
X = datafinal.drop('Years DM', axis = 1)
y = datafinal['Years DM']

In [64]:
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X, y, test_size=0.30, random_state= 42)

In [65]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

In [66]:
#Sklearn does't have any function for SMAPE, so I wrote a function in python
#The function has 100%/n. I replaced 100% with 1, to have values between 0 and 1 in form of percentages.
#A is the real, while F is predicted.
def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f)))

In [67]:
r2mean = []
SMAPEm = []
MSEm = []
RMSEm = []
MAEm= []

In [68]:
randomforest = RandomForestRegressor()

In [69]:
for x in range(1000): 
  X_train, X_test2, y_train, y_test2 = train_test_split(X, y, test_size=0.3)
  randomforest.fit(X_train, y_train)
  r2 = randomforest.score(X_test, y_test)
  y_pred = randomforest.predict(X_test)
  r2mean.append(r2)
  MAE = mean_absolute_error(y_test, y_pred)
  MAEm.append(MAE)
  MSE = mean_squared_error(y_test, y_pred)
  MSEm.append(MSE)
  RMSE = sqrt(mean_squared_error(y_test, y_pred))
  RMSEm.append(RMSE)
  SMAPE = smape(y_test, y_pred)
  SMAPEm.append(SMAPE)

In [70]:
Metrics = {'Metrics Means': ['R2', 'MSE', 'RMSE', 'SMAPE', 'MAE'],
           'Values': [np.mean(r2mean), np.mean(MSEm), np.mean(RMSEm), np.mean(SMAPEm), np.mean(MAE)]
           }

In [71]:
MetricsDF = pd.DataFrame.from_dict(Metrics)
MetricsDF

,Metrics Means,Values
0,R2,0.018201
1,MSE,66.325506
2,RMSE,8.136897
3,SMAPE,0.396052
4,MAE,5.525156


## Xgboost

In [72]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
xgb = XGBRegressor()

C:\Users\54226\Anaconda3\envs\python-ml-2020\lib\site-packages\xgboost\__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


In [73]:
r2mean = []
SMAPEm = []
MSEm = []
RMSEm = []
MAEm= []

In [74]:
for x in range(1000): 
  X_train, X_test2, y_train, y_test2 = train_test_split(X, y, test_size=0.30)
  xgb.fit(X_train, y_train)
  r2 = xgb.score(X_test, y_test)
  y_pred = xgb.predict(X_test)
  r2mean.append(r2)
  MAE = mean_absolute_error(y_test, y_pred)
  MAEm.append(MAE)
  MSE = mean_squared_error(y_test, y_pred)
  MSEm.append(MSE)
  RMSE = sqrt(mean_squared_error(y_test, y_pred))
  RMSEm.append(RMSE)
  SMAPE = smape(y_test, y_pred)
  SMAPEm.append(SMAPE)

In [75]:
Metrics = {'Metrics Means': ['R2', 'MSE', 'RMSE', 'SMAPE', 'MAE'],
           'Values': [np.mean(r2mean), np.mean(MSEm), np.mean(RMSEm), np.mean(SMAPEm), np.mean(MAE)]
           }

In [76]:
MetricsDF = pd.DataFrame.from_dict(Metrics)
MetricsDF

,Metrics Means,Values
0,R2,-0.020183
1,MSE,68.918568
2,RMSE,8.280922
3,SMAPE,0.405089
4,MAE,6.024323
